# The Synthetic Radio Host 📻

Turn any Wikipedia topic into a lively Hinglish podcast!

**Features:**
- **Source**: Wikipedia API
- **Script**: Groq (Llama 3.1 70B) for natural Hinglish dialogue
- **Audio**: Edge-TTS for high-quality Hindi voices
- **Output**: MP3 Podcast

In [ ]:
# Install dependencies
!pip install groq wikipedia-api edge-tts pydub python-dotenv nest_asyncio

In [ ]:
import os
import json
import asyncio
import nest_asyncio
import wikipediaapi
from groq import Groq
import edge_tts
from pydub import AudioSegment
from dotenv import load_dotenv

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    GROQ_API_KEY = input("Enter your Groq API Key: ")

In [ ]:
# Configuration

# TTS Voices (Edge-TTS)
VOICE_FEMALE = "hi-IN-SwaraNeural"  # Speaker 1 (Priya)
VOICE_MALE = "hi-IN-MadhurNeural"   # Speaker 2 (Amit)

# Output Directory
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# 1. Wikipedia Fetcher

def fetch_wikipedia_content(topic, lang='en'):
    """
    Fetches the summary and first few sections of a Wikipedia article.
    """
    wiki_wiki = wikipediaapi.Wikipedia(
        user_agent='SyntheticRadioPost/1.0 (contact@example.com)',
        language=lang
    )
    page = wiki_wiki.page(topic)

    if not page.exists():
        return None

    # Get summary and first 1500 chars of content to fit in context
    content = f"Title: {page.title}\n\nSummary: {page.summary}\n\n"
    
    # Add a bit more detail from sections
    content += "Details:\n"
    for section in page.sections[:3]: # First 3 sections
        content += f"\n{section.title}\n{section.text[:500]}...\n"
        
    return content[:4000] # Limit total length

In [ ]:
# 2. LLM Script Generator (Groq)

def generate_conversation_script(topic_content):
    """
    Generates a Hinglish conversation script using Groq (Llama 3.1).
    """
    client = Groq(api_key=GROQ_API_KEY)
    
    system_prompt = """
    You are a scriptwriter for a popular Indian Hinglish podcast called "Desi Talks".
    Hosts:
    1. RJ Priya (Female): Energetic, uses Gen-Z slang, mixes Hindi/English fluidly.
    2. RJ Amit (Male): Witty, slightly sarcastic, knowledgeable.

    Goal: Create a natural-sounding, 2-minute conversation script about the provided topic.

    Style Guidelines:
    - Language: Hinglish (Natural blend of Hindi and English).
    - Tone: Fun, conversational, informal (Radio Jockey style).
    - Markers: Use fillers like "Arre yaar", "Sach mein?", "You know", "Matlab", "Haina?".
    - Flow: Include interruptions, laughter [laughs], and thinking pauses [umm...].
    - Structure: The output MUST be a JSON list of objects. Each object has "speaker" (Priya/Amit) and "text" (the dialogue).
    
    Example Output Format:
    [
        {"speaker": "Priya", "text": "Hello Mumbai! Welcome back to Desi Talks. Aaj hum baat karenge ek special topic pe."},
        {"speaker": "Amit", "text": "Arre suspense mat banao yaar, bata do sabko!"}
    ]
    """
    
    user_prompt = f"Topic Content:\n{topic_content}\n\nGenerate the conversation script now."

    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        response_format={"type": "json_object"}
    )
    
    try:
        response_content = completion.choices[0].message.content
        # Ensure we get the JSON part if there's extra text (though json_object mode handles this mostly)
        script_data = json.loads(response_content)
        # Handle cases where the LLM might wrap the list in a key like "script"
        if isinstance(script_data, dict):
            # Look for a list value
            for key, value in script_data.items():
                if isinstance(value, list):
                    return value
            # If no list found, maybe the dict itself is the structure? Unlikely prompt asked for list.
            return script_data.get("conversation", script_data.get("script", []))
        return script_data
    except Exception as e:
        print(f"Error parsing LLM response: {e}")
        return []

In [ ]:
# 3. TTS Synthesizer (Edge-TTS)

async def generate_audio_segment(text, voice, filename):
    """
    Generates audio for a single segment using Edge-TTS.
    """
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(filename)

async def synthesize_podcast(script, output_file):
    """
    Converts the entire script to a single audio file.
    """
    temp_files = []
    combined_audio = AudioSegment.empty()
    
    print("Synthesizing audio segments...")
    
    for i, line in enumerate(script):
        speaker = line.get("speaker", "").lower()
        text = line.get("text", "")
        
        # Determine voice
        if "priya" in speaker:
            voice = VOICE_FEMALE
            print(f"Speaker: Priya (Female) - {text[:30]}...")
        else:
            voice = VOICE_MALE
            print(f"Speaker: Amit (Male) - {text[:30]}...")
            
        # Clean text of non-spoken cues for TTS (simple regex or replace)
        spoken_text = text.replace("[laughs]", "").replace("[umm...]", "umm").replace("[interrupts]", "")
        
        filename = f"{OUTPUT_DIR}/seg_{i}.mp3"
        await generate_audio_segment(spoken_text, voice, filename)
        temp_files.append(filename)
        
        # Add to combined audio
        segment_audio = AudioSegment.from_mp3(filename)
        combined_audio += segment_audio
        
        # Add a short pause between speakers (randomize slightly for naturalness)
        combined_audio += AudioSegment.silent(duration=300) 
        
    # Save final output
    combined_audio.export(output_file, format="mp3")
    print(f"\nPodcast saved to {output_file} ✅")
    
    # Cleanup temp files
    for f in temp_files:
        if os.path.exists(f):
            os.remove(f)
            
    return output_file

In [ ]:
# 4. Main Pipeline

async def run_synthetic_radio_host(topic):
    print(f"📻 Starting Synthetic Radio Host for topic: {topic}")
    
    # 1. Fetch Content
    print("Titl: Fetching Wikipedia content...")
    content = fetch_wikipedia_content(topic)
    if not content:
        print("Error: Topic not found on Wikipedia.")
        return
        
    # 2. Generate Script
    print("Writing script (this may take a few seconds)...")
    script = generate_conversation_script(content)
    print(f"Generated {len(script)} dialogue lines.")
    
    # 3. Synthesize Audio
    output_filename = f"{OUTPUT_DIR}/{topic.replace(' ', '_').lower()}.mp3"
    await synthesize_podcast(script, output_filename)
    
    return output_filename

In [ ]:
# Run the Host!
topic = "Mumbai Indians"
output = await run_synthetic_radio_host(topic)

In [ ]:
# Listen to the Result
from IPython.display import Audio
Audio(output)